# notebook n01a: Process and Visualize bodypart location

Jose Oliveira da Cruz, PhD  | LeDoux Lab  
jose.cruz@nyu.edu  

<img src="https://misophoniainternational.com/wp-content/uploads/2016/04/LedouxLab.jpg" style="width: 200.464px; height: 200px; margin: 0px;">

This notebooks takes raw datafiles (coord x, y) from deeplabcut and tranform the data to:

Step A: Create ``_bodypart_coordinates_dlc.csv``
1. Load data
2. Fetch Animal information
3. Extract interpolated bodypart coordinates for each cs and for each cs_epoch
4. Plot animal bodypart trajectory

Step B: Visualize bodypart trajectories

In [ ]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

# Import my code
sys.path.append(r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\src')
from tools.utils.organization import *
from analysis.motion_analysis import *
from analysis.freezing_analysis import *
from visualization import *
from visualization.plot_events import *

## Specify where to save the data for step A

In [ ]:
step_a_save_dir = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\interim\EXP004\bodypart_coordinates_dlc'

print(f'Does the directory exist? \n a: {os.path.isdir(step_a_save_dir)}') 

# Step A: ``_bodypart_coordinates_dlc.csv``

## 1) Load **individual** raw data from deeplabcut

### 1.1) Where is the file?

In [ ]:
# Open dataframe

dpath = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\interim\EXP004\deeplabcut_pose_extraction'
fpath = 'JC_EXP004_20200110_TES01_R_286600_T00DLC_resnet50_threat_conditioningMay18shuffle1_300000.h5'

print(f'File exists?\n- {os.path.isfile(os.path.join(dpath, fpath))}')

### 1.2) Open the file

In [ ]:
# Load data
df = pd.read_hdf(
    os.path.join(dpath, fpath), 
    header=[1, 2],
)
#Inspect dataframe
df.head(5)

## 2) Fetch information about a specific rat

The code bellow will read the video key( e.g `JC_EXP005_20200124_TES01_R_287073_T00`) and search for the complete information about this specific animal.  
Then it creates an instance of the Animal class with the complete information of the animal.

### 2.1) Load Global Animal Record and Experiment information

In [ ]:
# Where is the main record?
main_record_directory = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\animal_record\main_record'
main_record_basename = 'main_record_20200325_151527.csv'

main_record = os.path.join(main_record_directory,
                           main_record_basename)

# Where is the information about the experiment?
experiment_info_directory = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\interim\EXP004\bonsai_extraction_led_epochs_frame_rate'
experiment_info_basename = 'JC_EXP004_20200110_TES01_cs_index_plus_frame_rate.csv'
experiment_info = os.path.join(experiment_info_directory,
                               experiment_info_basename)

### 2.2) Fetch animal information

In [ ]:
# Search object with regular expressions
pattern = r'(\w\w_\w\w\w\d\d\d_\d\d\d\d\d\d\d\d_\w\w\w\d\d_\w_\d\d\d\d\d\d_\w\d\d)'
video_key = re.search(pattern, fpath).group()

# Fetch information
rat = fetch_animal_info(
    video_key, 
    main_record, 
    experiment_info,
)

## 3) Extract interpolated bodypart coordinates for each cs and for each cs_epoch

### 3.1) Extract interpolated tracking location
The code bellow extracts the x, y position for each bodypart, during each cs_id and cs_epoch. In addition, it corrects aquisition rate for 30fps. 

In [ ]:
positions_dataframe = create_bodypart_coord_dataframe(df.copy(), rat)

In [ ]:
positions_dataframe.head()

### 3.2) Save file

In [ ]:
saving_basename = f'{rat.video_basename}_bodypart_coordinates_dlc.csv'.lower()

positions_dataframe.to_csv(os.path.join(step_a_save_dir, saving_basename))

## 4) Plot position of the animal

The code bellows allow to plot the position of a specific bodypart during a specific cs(e.g. cs_01) and specific epoch (eg peri_cs > during cs).  

`fetch_bodypart_coordinates()` allows to fetch x, y coordinates for a specific bodypart using the above created dataframe.  To check the information doc associated with the function you can use:  
- `fetch_bodypart_coordinates?`

In [ ]:
fetch_bodypart_coordinates?

### 4.1) Plot trajectory of bodypart before, during and after cs

In [ ]:
fig_trajectories, ax = plt.subplots(figsize=[5, 5])

cs_epochs = ['pre_cs', 'post_cs', 'peri_cs']
for cs_epoch in cs_epochs:
      plot_bodypart_trajectory(
          ax=ax,
          dataframe=positions_dataframe,
          bodypart='upper_torso',
          cs_id='cs_01',
          cs_epoch=cs_epoch,
          animal=rat,
      )

plt.show()

In [ ]:
fig_trajectories, ax = plt.subplots(figsize=[5, 5])

number_of_cs = 5

cs_id_list = [f'cs_0{number}' for number in range(1, number_of_cs+1)]

for cs_id in cs_id_list:
      plot_bodypart_trajectory(
          ax=ax,
          dataframe=positions_dataframe,
          bodypart='upper_torso',
          cs_id=cs_id,
          cs_epoch='peri_cs',
          animal=rat,
      )

plt.show()

### Save fig

`fname` = absolute path > where do you want the figure to be saved. Must include extension (e.g. `.pdf`)
example:  
`fname=C:\Users\jfo2\Desktop\test_data\figure.pdf`


In [ ]:
#fig_trajectories.savefig(
#    fname=,
#    bbox_inches='tight',
#)

### 4.3) Try other combinations by changing the arguments of the function

In [ ]:
fig_trajectories, ax = plt.subplots(figsize=[5, 5])

plot_bodypart_trajectory(
    ax=ax,
    dataframe=positions_dataframe,
    bodypart='upper_torso',
    cs_id='cs_01',
    cs_epoch='peri_cs',
    animal=rat,
)

plt.show()